# **Gobest Cab – Sprint 2 (CA2): ML Model for Trip Safety Prediction**
**Role:** Person A (ML Engineer)  
**Goal:** Train a machine learning model to classify trips as **Safe (0)** or **Dangerous (1)** using sensor data.

**Deliverables for Sprint 2:**
1. ML-focused EDA
2. Advanced Feature Engineering (≥10 new features, no leakage)
3. Model training + evaluation (and later deployment integration)
4. Track experiments (e.g., MLflow)


---

#### SPRINT 1 vs SPRINT 2 FEATURE ENGINEERING:

Sprint 1 (Dashboard-focused):
   - Simple aggregations:  avg, max, count, sum
   - Descriptive metrics for visualization
   - Example: avg_speed_kmh, harsh_brake_count

Sprint 2 (ML Prediction-focused):
   - Complex temporal patterns: rolling windows, burstiness
   - Statistical distributions: percentiles, skewness
   - Interactions: combined behaviors
   - Example: max_rolling_30s_speed_variance, high_speed_brake_interaction

---

#### FEATURE DESIGN PRINCIPLES:

1. Domain Knowledge:  What makes driving dangerous?

   → Erratic steering, sudden braking, speed volatility

<br>

2. EDA Insights: What differs between safe/dangerous trips?

   → Gyro_z_std is +58.6% higher in dangerous trips

   → Dangerous trips have LOWER avg speed but HIGHER volatility

   → Urban "stop-and-go" aggression is key pattern

<br>

3. No Data Leakage: Features must be computable at prediction time

   → Only use data available within current trip

   → Driver history uses ONLY past trips (time-aware split later)

<br>

4. Temporal Focus: Capture short-term bursts of risky behavior

   → 30s rolling windows (burst detection)

   → Consecutive event counting (sustained risk)

---

GOAL: Create at least 10 advanced features across these categories: 

   1. Rolling-window features (burst behavior)

   2. Distribution features (extremes & percentiles)

   3. Burstiness metrics (consecutive events)
   
   4. Interaction features (combined behaviors)

   5. Data quality features (sensor reliability)

   6. Driver-level features (experience, demographics)

---

**STEP 0: SETUP**

In [48]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from scipy import stats

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("="*70)
print("ADVANCED FEATURE ENGINEERING - SPRINT 2")
print("="*70)
print(f"Start Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print()

ADVANCED FEATURE ENGINEERING - SPRINT 2
Start Time: 2026-01-22 01:36:28



In [49]:
df = pd.read_pickle("datasets/sensor_data_preprocessed.pkl")

print("Data loaded successfully!")
print(f"\nDataset shape: {df.shape}")
print(f"Unique trips: {df['bookingID'].nunique():,}")
print(f"Unique drivers: {df['driver_id'].nunique():,}")

Data loaded successfully!

Dataset shape: (7197740, 44)
Unique trips: 19,923
Unique drivers: 500


---

**CATEGORY 1: ROLLING WINDOW FEATURES**

`PURPOSE`: Capture SHORT-TERM BURSTS of risky behavior

`WHY ROLLING WINDOWS?`
- Dangerous driving often happens in bursts (sudden aggressive maneuvers)
- Trip-level averages MISS these bursts
- Example: A driver might be calm for 10 minutes, then aggressive for 30 seconds
- Rolling windows capture the WORST 30-second period of the trip

`FEATURES TO CREATE:`
1. speed_roll30_mean_max:  Max 30-second average speed
   → Captures sustained high-speed bursts (not just 1-second spikes)

2. speed_roll30_std_max: Max 30-second speed volatility
   → Captures most erratic 30-second window

3. acc_roll10_std_max: Max 10-second longitudinal acceleration volatility
   → Captures most aggressive braking/acceleration burst

4. gyro_z_roll10_absmax_max: Max 10-second peak gyro_z (steering intensity)
   → Captures sharpest turning burst (gyro_z is strongest predictor!)

`WHY THESE WINDOW SIZES?`
- 30 seconds: Typical duration of a dangerous maneuver (speeding, weaving)
- 10 seconds: Captures individual events (harsh brake, sharp turn)

`EDA JUSTIFICATION:`
- Our EDA showed gyro_z_std is +58.6% higher in dangerous trips
- Distribution overlap means we need to capture PEAK behavior, not just averages

In [50]:

# Feature 1: Max 30-second average speed
print("\n1) speed_roll30_mean_max (max 30s avg speed)")

df['speed_roll30_mean'] = (
    df.groupby('bookingID')['Speed']
    .transform(lambda x: x.rolling(window=30, min_periods=1).mean())
)

# Feature 2: Max 30-second speed volatility
print("2) speed_roll30_std_max (max 30s speed volatility)")

df['speed_roll30_std'] = (
    df.groupby('bookingID')['Speed']
      .transform(lambda x: x.rolling(30, min_periods=1).std(ddof=0))
)

# Feature 3: Max 10-second acceleration volatility
print("3) acc_roll10_std_max (max 10s acceleration volatility)")

df['acc_roll10_std'] = (
    df.groupby('bookingID')['longitudinal_acc']
    .transform(lambda x: x.rolling(window=10, min_periods=1).std(ddof=0))
)

# Feature 4: Max 10-second gyro_z peak (steering intensity)
print("4) gyro_z_roll10_absmax_max (max 10s gyro_z peak)")

df['gyro_z_roll10_absmax'] = (
    df.groupby('bookingID')['gyro_z']
    .transform(lambda x: x.rolling(window=10, min_periods=1)
               .apply(lambda y: np.max(np.abs(y)), raw=True))
)

print("\nRolling window features computed!")

# Quick verification
print("\nRolling feature statistics:")
for col in ['speed_roll30_mean', 'speed_roll30_std', 'acc_roll10_std', 'gyro_z_roll10_absmax']:
    print(f"   {col}:  min={df[col].min():.2f}, max={df[col].max():.2f}, "
          f"NaNs={df[col].isnull().sum()}")


1) speed_roll30_mean_max (max 30s avg speed)
2) speed_roll30_std_max (max 30s speed volatility)
3) acc_roll10_std_max (max 10s acceleration volatility)
4) gyro_z_roll10_absmax_max (max 10s gyro_z peak)

Rolling window features computed!

Rolling feature statistics:
   speed_roll30_mean:  min=0.00, max=37.65, NaNs=0
   speed_roll30_std:  min=0.00, max=18.86, NaNs=0
   acc_roll10_std:  min=0.00, max=18.89, NaNs=0
   gyro_z_roll10_absmax:  min=0.00, max=53.55, NaNs=0


Rolling window features are `first computed at the sensor level` to capture short-term bursts of risky behaviour. These rolling statistics are then `aggregated to the trip level by taking the maximum value`, representing the most dangerous period within each trip. This allows the model to learn from peak driving risk rather than averaged behaviour.

---

**CATEGORY 2: DISTRIBUTION FEATURES**

`PURPOSE`:` Capture EXTREME behaviors and statistical distributions

`WHY PERCENTILES INSTEAD OF MAX?`

- Max values are sensitive to GPS errors and outliers

- 95th/99th percentiles are more robust

- Example: If max_speed is 180 km/h due to GPS error, p99 still accurate

`FEATURES TO CREATE`:

5. speed_p95:  95th percentile speed

   → "Typical high speed" (ignores rare spikes)

<br>

6. speed_p99: 99th percentile speed

   → Captures sustained high-speed behavior

<br>

7. longitudinal_acc_min: Most intense braking

   → Captures hardest single brake event

<br>

8. longitudinal_acc_max: Most intense acceleration

   → Captures hardest single acceleration event

<br>

9. speed_std:  Speed standard deviation

   → Overall trip volatility (complements rolling windows)
   
<br>

10. gyro_z_abs_p95: 95th percentile of |gyro_z|

    → Typical sharp turn intensity

`EDA JUSTIFICATION:`
- Our EDA showed dangerous trips have LOWER avg_speed but HIGHER volatility
- Percentiles capture "how often does driver behave dangerously?" 
  (not just "what's the single worst moment? ")

<mark> IMPORTANT NOTE </mark>

- These features are `computed PER TRIP` (not per row).

- They require ALL data points from a trip to calculate statistics.

- Therefore, they are computed in `PART 4 (Trip-Level Aggregation)`.

`PART 1 (Rolling Windows)` = Row-by-row computation, then aggregate MAX

`PART 2 (Distributions)` = Direct aggregation during groupby


---

**CATEGORY 3: BURSTINESS METRICS**

`PURPOSE`: Capture SUSTAINED risky behavior (not isolated incidents)

`WHY CONSECUTIVE EVENTS MATTER?`

- Single harsh brake = might be unavoidable (pedestrian, traffic light)

- 5 consecutive harsh brakes = aggressive driving pattern

- Measures "loss of control" vs "one-time reaction"

`FEATURES TO CREATE:` 
11. max_consecutive_speeding_secs:  Longest speeding burst

    → Distinguishes brief passing (5s) from sustained speeding (100s)

<br>

12. max_consecutive_harsh_brakes: Most harsh brakes in a row

    → Detects "panic braking" or "tailgating" patterns
    
<br>

13. max_consecutive_sharp_turns:  Consecutive sharp turns

    → Detects "weaving through traffic" behavior
    
<br>

`EDA JUSTIFICATION:`
- Our EDA showed event COUNTS are +40% higher in dangerous trips
- But GAPS between events were LONGER (counterintuitive!)
- This suggests:  focus on INTENSITY of bursts, not just frequency
- Consecutive events capture "clusters of aggression"

METHODOLOGY:
- Group by bookingID
- Identify consecutive True values in event flags
- Count maximum consecutive sequence

In [55]:
# Ensure flags are clean 0/1 (no NaNs)
for flag in ['is_speeding', 'is_harsh_braking', 'is_sharp_turn']: 
    if df[flag].isnull().any():
        print(f"   {flag} has NaNs. Filling with 0...")
        df[flag] = df[flag].fillna(0).astype(int)


# Helper Function: Count Maximum Consecutive 1s
def max_consecutive_ones(series):
    """
    Count maximum consecutive True/1 values in a series.
    
    Example: 
        [0, 1, 1, 1, 0, 1, 1, 0] → returns 3
        [0, 0, 0, 0] → returns 0
        [1, 1, 1, 1, 1] → returns 5
    
    How it works:
    1. Ensure series is 0/1 integers (no NaNs)
    2. Create run IDs:   each time value changes, new run
    3. Within each run of 1s, sum = length of run
    4. Return max run length
    """
    series = series.fillna(0).astype(int)
    
    if series.sum() == 0:
        return 0
    
    # Create run groups (increments when value changes)
    runs = (series != series.shift()).cumsum()
    
    # Sum within each run (for runs of 1s, sum = length)
    consecutive_counts = series.groupby(runs).sum()
    
    return consecutive_counts.max()

# Compute Burstiness Features (Direct Aggregation)
print("\n   Computing burstiness metrics (this may take 1-2 minutes)...")

burstiness_features = df.groupby('bookingID').agg(
    max_consec_speeding=('is_speeding', max_consecutive_ones),
    max_consec_harsh_brakes=('is_harsh_braking', max_consecutive_ones),
    max_consec_sharp_turns=('is_sharp_turn', max_consecutive_ones),
).reset_index()

print("\nBurstiness metrics computed!")

# Verification
print("\nBurstiness Feature Statistics:")
for col in ['max_consec_speeding', 'max_consec_harsh_brakes', 'max_consec_sharp_turns']: 
    print(f"   {col}:")
    print(f"      Mean:   {burstiness_features[col].mean():.2f}")
    print(f"      Max:  {burstiness_features[col].max():.0f}")
    print(f"      % trips with 0:  {(burstiness_features[col]==0).mean()*100:.1f}%")

# Sample output
print("\nSample burstiness features:")
print(burstiness_features.head(10))


   Computing burstiness metrics (this may take 1-2 minutes)...

Burstiness metrics computed!

Burstiness Feature Statistics:
   max_consec_speeding:
      Mean:   4.79
      Max:  291
      % trips with 0:  76.3%
   max_consec_harsh_brakes:
      Mean:   0.49
      Max:  4
      % trips with 0:  53.0%
   max_consec_sharp_turns:
      Mean:   1.19
      Max:  614
      % trips with 0:  47.9%

Sample burstiness features:
   bookingID  max_consec_speeding  max_consec_harsh_brakes  \
0          0                    0                        1   
1          1                    0                        1   
2          2                    0                        0   
3          4                    0                        1   
4          6                    0                        0   
5          7                    1                        0   
6          8                    0                        1   
7         10                    0                        0   
8         11      

---

**STEP 4: AGGREGATE TO TRIP-LEVEL FEATURE TABLE**

`PURPOSE`: Create ONE ROW PER TRIP with all features

`AGGREGATION STRATEGY:`
- Rolling windows: Take MAX (worst 30-second window)
- Event counts: SUM (total events in trip)
- Distributions:  Percentiles, std, min, max
- Flags:  MEAN (% of trip with issue)
- Driver info:  FIRST (same for all rows in trip)

`FEATURE CATEGORIES IN FINAL TABLE:`
1. Trip metadata:  duration, bookingID
2. Basic speed metrics: mean, max, std, p95, p99
3. Acceleration metrics: min, max, std
4. Gyro metrics: std, abs_p95, max
5. Rolling window maxes: speed_roll30_mean_max, etc.
6. Event counts: speeding, harsh_brake, harsh_accel, sharp_turn
7. Burstiness: max_consecutive metrics
8. Interaction features: (computed after aggregation)
9. Data quality: GPS quality %, sensor missing %
10. Driver features: age, experience, rating, gender
11. Target: label

In [58]:
trip_features_ml = df.groupby('bookingID').agg({

    # 1. TRIP METADATA
    'second': lambda x: x.max() - x.min(),  # trip_duration
    

    # 2. SPEED FEATURES
    'Speed':  [
        'mean',  # avg_speed
        'max',   # max_speed
        'std',   # speed_std (Feature 9)
        lambda x: x.quantile(0.95),  # speed_p95 (Feature 5)
        lambda x: x.quantile(0.99),  # speed_p99 (Feature 6)
    ],
    
    # 3. LONGITUDINAL ACCELERATION
    'longitudinal_acc': [
        'mean',
        'std',
        'min',  # longitudinal_acc_min (Feature 7)
        'max',  # longitudinal_acc_max (Feature 8)
    ],
    
    # 4. GYROSCOPE (STEERING)
    'gyro_z':  [
        'std',  # gyro_z_std (strongest predictor!)
        'max',
        lambda x: np.abs(x).quantile(0.95),  # gyro_z_abs_p95 (Feature 10)
    ],
    
    # 5. ROLLING WINDOW MAXES
    'speed_roll30_mean':   'max',  # Feature 1
    'speed_roll30_std':  'max',   # Feature 2
    'acc_roll10_std': 'max',     # Feature 3
    'gyro_z_roll10_absmax': 'max',  # Feature 4
    
    # 6. EVENT COUNTS
    'is_speeding': 'sum',  # speeding_duration
    'is_harsh_braking': 'sum',  # harsh_brake_count
    'is_harsh_acceleration': 'sum',  # harsh_accel_count
    'is_sharp_turn': 'sum',  # sharp_turn_count
    'is_stationary': 'mean',  # stationary_pct
    
    # 7. DATA QUALITY
    'low_gps_quality_flag': 'mean',  # gps_quality_pct
    'acc_x_missing_flag': 'mean',  # acc_x_missing_pct
    
    # 8. DRIVER FEATURES
    'age': 'first',
    'No_of_Years_driving_exp': 'first',
    'driver_rating': 'first',
    'gender': 'first',
    'car_model_year': 'first',
    'driver_id': 'first',
    
    # 9. TARGET
    'label': 'first',
    
}).reset_index()

# Flatten column names
trip_features_ml.columns = ['_'.join(col).strip('_') if col[1] else col[0] 
                         for col in trip_features_ml.columns]

In [59]:
# MERGE BURSTINESS FEATURES (computed separately)
print("\nMerging burstiness features...")

trip_features_ml = trip_features_ml.merge(
    burstiness_features,
    on='bookingID',
    how='left'
)

print(f"\nTrip-level aggregation complete!")
print(f"   Shape: {trip_features_ml.shape}")
print(f"   Trips: {trip_features_ml.shape[0]: ,}")
print(f"   Features: {trip_features_ml.shape[1]}")


Merging burstiness features...

Trip-level aggregation complete!
   Shape: (19923, 35)
   Trips:  19,923
   Features: 35


In [61]:
# confirming same number of trips as in df
print(df['bookingID'].nunique(), trip_features_ml['bookingID'].nunique())

19923 19923


In [63]:
# confirming no missing burstiness values
print(trip_features_ml[['max_consec_speeding','max_consec_harsh_brakes','max_consec_sharp_turns']].isnull().sum())

max_consec_speeding        0
max_consec_harsh_brakes    0
max_consec_sharp_turns     0
dtype: int64


---

**STEP 5: RENAME & CLEAN AGGREGATED FEATURES**

`PURPOSE:` Convert pandas-generated column names to readable names

`PANDAS AGGREGATION NAMING: `
- Lambda functions → 'column_<lambda_0>', 'column_<lambda_1>'
- Aggregation functions with 'first' → 'column_first'
- Need to rename for clarity and consistency

`CLEANING STEPS:`
1. Rename all aggregated columns
2. Handle any NaNs from aggregation (e.g., std of single-row trips)
3. Verify data types
4. Check for any unexpected issues

In [65]:
trip_features_ml.columns

Index(['bookingID', 'second_<lambda>', 'Speed_mean', 'Speed_max', 'Speed_std',
       'Speed_<lambda_0>', 'Speed_<lambda_1>', 'longitudinal_acc_mean',
       'longitudinal_acc_std', 'longitudinal_acc_min', 'longitudinal_acc_max',
       'gyro_z_std', 'gyro_z_max', 'gyro_z_<lambda_0>',
       'speed_roll30_mean_max', 'speed_roll30_std_max', 'acc_roll10_std_max',
       'gyro_z_roll10_absmax_max', 'is_speeding_sum', 'is_harsh_braking_sum',
       'is_harsh_acceleration_sum', 'is_sharp_turn_sum', 'is_stationary_mean',
       'low_gps_quality_flag_mean', 'acc_x_missing_flag_mean', 'age_first',
       'No_of_Years_driving_exp_first', 'driver_rating_first', 'gender_first',
       'car_model_year_first', 'driver_id_first', 'label_first',
       'max_consec_speeding', 'max_consec_harsh_brakes',
       'max_consec_sharp_turns'],
      dtype='object')

In [66]:
trip_features_ml

,bookingID,second_<lambda>,Speed_mean,Speed_max,Speed_std,Speed_<lambda_0>,Speed_<lambda_1>,longitudinal_acc_mean,longitudinal_acc_std,longitudinal_acc_min,longitudinal_acc_max,gyro_z_std,gyro_z_max,gyro_z_<lambda_0>,speed_roll30_mean_max,speed_roll30_std_max,acc_roll10_std_max,gyro_z_roll10_absmax_max,is_speeding_sum,is_harsh_braking_sum,is_harsh_acceleration_sum,is_sharp_turn_sum,is_stationary_mean,low_gps_quality_flag_mean,acc_x_missing_flag_mean,age_first,No_of_Years_driving_exp_first,driver_rating_first,gender_first,car_model_year_first,driver_id_first,label_first,max_consec_speeding,max_consec_harsh_brakes,max_consec_sharp_turns
0,0,1587.0,9.092575,22.882523,7.024853,19.578587,21.860323,0.037033,0.923695,-4.799928,12.102637,0.061070,0.338149,0.112720,20.201998,6.518371,4.033323,0.338149,0,2,4,3,0.000000,0.000000,0.013605,55,9,4.7,Female,2003,359,0,0,1,1
1,1,1034.0,7.613304,21.882141,6.921849,20.154978,21.478647,0.014883,0.948796,-9.307960,5.762546,0.032432,0.136467,0.074784,20.342373,6.835695,2.890585,0.136467,0,2,1,0,0.007792,0.000000,0.018182,45,24,4.2,Female,2000,313,1,0,1,0
2,2,812.0,2.927494,9.257438,2.794407,7.591568,9.190624,-0.014966,0.692940,-2.114535,2.139548,0.033069,0.112942,0.076698,5.228102,3.207009,1.158372,0.112942,0,0,0,0,0.000000,0.000000,0.000000,55,25,3.0,Male,2009,27,1,0,0,0
3,4,1091.0,5.989352,19.559999,5.390686,17.050000,18.280001,0.000982,0.614727,-3.010000,2.610000,0.064847,0.505220,0.137543,14.528333,7.215788,1.288021,0.505220,0,1,0,1,0.000000,0.000000,0.023355,46,24,4.8,Female,1989,164,1,0,1,1
4,6,1091.0,4.791179,16.394695,5.348769,14.210676,15.868304,0.006889,0.625879,-2.358797,3.011751,0.059123,0.258855,0.120036,13.510656,7.420576,1.274761,0.336128,0,0,1,3,0.000000,0.000000,0.023715,47,18,3.9,Female,1974,118,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19918,1709396983957,1149.0,2.441287,6.275580,1.736811,4.938154,5.948795,0.002785,0.180579,-1.200673,1.351471,0.382367,1.884930,0.795380,5.238570,1.926164,0.523665,2.603097,0,0,0,136,0.027149,0.000000,0.027149,49,12,3.3,Female,2000,112,1,0,0,7
19919,1709396983960,807.0,7.487312,24.059151,6.951016,20.786285,23.656548,0.034350,0.956755,-7.392830,4.473341,0.050791,0.233631,0.106669,20.406973,10.020894,2.318538,0.496440,0,3,2,1,0.000000,0.000000,0.024316,55,7,2.8,Female,1989,311,1,0,1,1
19920,1709396983966,987.0,12.729189,25.640000,7.525010,22.656000,25.147200,-0.003995,0.629198,-4.030000,2.615000,0.042867,0.151810,0.086197,20.902333,5.615198,1.421873,0.252991,6,1,0,1,0.024024,0.000000,0.015015,52,6,2.8,Male,1997,426,1,2,1,1
19921,1709396983971,1075.0,6.120885,19.287226,5.717626,15.113518,17.134327,-0.006572,0.491536,-1.851122,1.643064,0.076225,0.929654,0.133024,14.417082,7.208729,0.922752,0.929654,0,0,0,6,0.000000,0.000000,0.023555,52,21,4.7,Female,2011,139,1,0,0,1


In [67]:
# rename for clarity
rename_dict = {
    # Trip metadata
    'second_<lambda>': 'trip_duration',

    # Speed
    'Speed_mean': 'avg_speed',
    'Speed_max': 'max_speed',
    'Speed_std': 'speed_std',
    'Speed_<lambda_0>': 'speed_p95',
    'Speed_<lambda_1>': 'speed_p99',

    # Acceleration
    'longitudinal_acc_mean': 'long_acc_mean',
    'longitudinal_acc_std': 'long_acc_std',
    'longitudinal_acc_min': 'long_acc_min',
    'longitudinal_acc_max': 'long_acc_max',

    # Gyroscope
    'gyro_z_std': 'gyro_z_std',
    'gyro_z_max': 'gyro_z_max',
    'gyro_z_<lambda_0>': 'gyro_z_abs_p95',

    # Rolling windows
    'speed_roll30_mean_max': 'speed_roll30_mean_max',
    'speed_roll30_std_max': 'speed_roll30_std_max',
    'acc_roll10_std_max': 'acc_roll10_std_max',
    'gyro_z_roll10_absmax_max': 'gyro_z_roll10_absmax_max',

    # Events
    'is_speeding_sum': 'speeding_count',
    'is_harsh_braking_sum': 'harsh_brake_count',
    'is_harsh_acceleration_sum': 'harsh_accel_count',
    'is_sharp_turn_sum': 'sharp_turn_count',
    'is_stationary_mean': 'stationary_pct',

    # Data quality
    'low_gps_quality_flag_mean': 'gps_poor_quality_pct',
    'acc_x_missing_flag_mean': 'acc_x_missing_pct',

    # Driver features
    'age_first': 'age',
    'No_of_Years_driving_exp_first': 'driving_experience',
    'driver_rating_first': 'driver_rating',
    'gender_first': 'gender',
    'car_model_year_first': 'car_model_year',
    'driver_id_first': 'driver_id',

    # Target
    'label_first': 'label',
}

trip_features_ml = trip_features_ml.rename(columns=rename_dict)

In [ ]:
#ensure no missing values
missing_counts = trip_features_ml.isnull().sum()
print(missing_counts)

bookingID                   0
trip_duration               0
avg_speed                   0
max_speed                   0
speed_std                   0
speed_p95                   0
speed_p99                   0
long_acc_mean               0
long_acc_std                0
long_acc_min                0
long_acc_max                0
gyro_z_std                  0
gyro_z_max                  0
gyro_z_abs_p95              0
speed_roll30_mean_max       0
speed_roll30_std_max        0
acc_roll10_std_max          0
gyro_z_roll10_absmax_max    0
speeding_count              0
harsh_brake_count           0
harsh_accel_count           0
sharp_turn_count            0
stationary_pct              0
gps_poor_quality_pct        0
acc_x_missing_pct           0
age                         0
driving_experience          0
driver_rating               0
gender                      0
car_model_year              0
driver_id                   0
label                       0
max_consec_speeding         0
max_consec

In [ ]:
# Verify Data Types
print("\nData type distribution:")
dtype_counts = trip_features_ml.dtypes.value_counts()
for dtype, count in dtype_counts.items():
    print(f"   {dtype}: {count} columns")

# Check categorical columns
categorical_cols = trip_features_ml.select_dtypes(include=['object']).columns.tolist()
categorical_cols = [col for col in categorical_cols if col not in ['bookingID', 'driver_id']]

if categorical_cols:
    print(f"\nCategorical columns to encode later:")
    for col in categorical_cols:
        unique_vals = trip_features_ml[col].nunique()
        print(f"   - {col}: {unique_vals} unique values")


Data type distribution:
   float64: 21 columns
   int64: 12 columns
   int32: 1 columns
   object: 1 columns

Categorical columns to encode later:
   - gender: 2 unique values


In [72]:
# Final Column Inventory
# Categorize columns
identifiers = ['bookingID', 'driver_id']
target = ['label']
speed_features = [col for col in trip_features_ml.columns if 'speed' in col.lower() and col not in identifiers + target]
accel_features = [col for col in trip_features_ml.columns if 'acc' in col.lower() or 'long_' in col.lower()]
gyro_features = [col for col in trip_features_ml.columns if 'gyro' in col.lower()]
event_features = [col for col in trip_features_ml.columns if any(x in col for x in ['speeding_count', 'harsh_brake', 'harsh_accel', 'sharp_turn', 'consec'])]
quality_features = [col for col in trip_features_ml.columns if any(x in col for x in ['gps', 'missing', 'stationary'])]
driver_features = [col for col in trip_features_ml.columns if col in ['age', 'driving_experience', 'driver_rating', 'gender', 'car_model_year']]
metadata = ['trip_duration']

print(f"\nFEATURE BREAKDOWN:")
print(f"   Identifiers: {len(identifiers)} → {identifiers}")
print(f"   Target: {len(target)} → {target}")
print(f"   Metadata: {len(metadata)} → {metadata}")
print(f"   Speed features: {len(speed_features)}")
for f in speed_features: print(f"      - {f}")
print(f"   Acceleration features: {len(accel_features)}")
for f in accel_features:  print(f"      - {f}")
print(f"   Gyroscope features: {len(gyro_features)}")
for f in gyro_features: print(f"      - {f}")
print(f"   Event/Burstiness features: {len(event_features)}")
for f in event_features: print(f"      - {f}")
print(f"   Quality features: {len(quality_features)}")
for f in quality_features: print(f"      - {f}")
print(f"   Driver features: {len(driver_features)}")
for f in driver_features: print(f"      - {f}")


FEATURE BREAKDOWN:
   Identifiers: 2 → ['bookingID', 'driver_id']
   Target: 1 → ['label']
   Metadata: 1 → ['trip_duration']
   Speed features: 9
      - avg_speed
      - max_speed
      - speed_std
      - speed_p95
      - speed_p99
      - speed_roll30_mean_max
      - speed_roll30_std_max
      - speeding_count
      - max_consec_speeding
   Acceleration features: 7
      - long_acc_mean
      - long_acc_std
      - long_acc_min
      - long_acc_max
      - acc_roll10_std_max
      - harsh_accel_count
      - acc_x_missing_pct
   Gyroscope features: 4
      - gyro_z_std
      - gyro_z_max
      - gyro_z_abs_p95
      - gyro_z_roll10_absmax_max
   Event/Burstiness features: 7
      - speeding_count
      - harsh_brake_count
      - harsh_accel_count
      - sharp_turn_count
      - max_consec_speeding
      - max_consec_harsh_brakes
      - max_consec_sharp_turns
   Quality features: 3
      - stationary_pct
      - gps_poor_quality_pct
      - acc_x_missing_pct
   Driver feature

In [77]:
# Verify Data Types
print("\nData type distribution:")
dtype_counts = trip_features_ml.dtypes.value_counts()
for dtype, count in dtype_counts.items():
    print(f"   {dtype}: {count} columns")

# Check categorical columns
categorical_cols = trip_features_ml.select_dtypes(include=['object']).columns.tolist()
categorical_cols = [col for col in categorical_cols if col not in ['bookingID', 'driver_id']]

if categorical_cols:
    print(f"\nCategorical columns to encode later:")
    for col in categorical_cols:
        unique_vals = trip_features_ml[col].nunique()
        print(f"   - {col}: {unique_vals} unique values")


Data type distribution:
   float64: 25 columns
   int64: 12 columns
   int32: 1 columns
   object: 1 columns

Categorical columns to encode later:
   - gender: 2 unique values


---

**CATEGORY 4: INTERACTION FEATURES**

`PURPOSE`: Capture COMBINED risky behaviors

`WHY INTERACTIONS?`   
- Our EDA showed massive distribution overlap between classes
- No single feature separates safe from dangerous
- COMBINATIONS are key:  "High speed" + "Hard braking" = very dangerous

`FEATURES TO CREATE:`  
14. highspeed_brake_interaction: speed_p95 × harsh_brake_count

    → Braking at high speed is MORE dangerous than braking at low speed

15. speed_turn_interaction: avg_speed × sharp_turn_count

    → Sharp turns at high speed = loss of control risk

16. volatility_interaction: speed_std × gyro_z_std

    → Erratic speed + erratic steering = "out of control" pattern

17. aggressive_burst_score: speed_roll30_std_max × harsh_accel_count

    → Combines burst volatility with acceleration aggression

`EDA JUSTIFICATION:`
- Your proxy_risk_score (simple sum) performed WORSE than baseline
- Models need to learn complex interactions
- Explicit interaction features help tree-based models (XGBoost, RF)

In [75]:

# Feature 14: High-speed braking interaction
print("\n14) highspeed_brake_interaction")
trip_features_ml['highspeed_brake_interaction'] = (
    trip_features_ml['speed_p95'] * trip_features_ml['harsh_brake_count']
)


# Feature 15: Speed-turn interaction
print("15) speed_turn_interaction")
trip_features_ml['speed_turn_interaction'] = (
    trip_features_ml['avg_speed'] * trip_features_ml['sharp_turn_count']
)


# Feature 16: Volatility interaction (speed × steering)
print("16) volatility_interaction")
trip_features_ml['volatility_interaction'] = (
    trip_features_ml['speed_std'] * trip_features_ml['gyro_z_std']
)


# Feature 17: Aggressive burst score

print("17) aggressive_burst_score")
trip_features_ml['aggressive_burst_score'] = (
    trip_features_ml['speed_roll30_std_max'] * trip_features_ml['harsh_accel_count']
)



14) highspeed_brake_interaction
15) speed_turn_interaction
16) volatility_interaction
17) aggressive_burst_score


In [76]:
# Verify no NaNs introduced
null_counts = trip_features_ml.isnull().sum()
print(null_counts)

bookingID                      0
trip_duration                  0
avg_speed                      0
max_speed                      0
speed_std                      0
speed_p95                      0
speed_p99                      0
long_acc_mean                  0
long_acc_std                   0
long_acc_min                   0
long_acc_max                   0
gyro_z_std                     0
gyro_z_max                     0
gyro_z_abs_p95                 0
speed_roll30_mean_max          0
speed_roll30_std_max           0
acc_roll10_std_max             0
gyro_z_roll10_absmax_max       0
speeding_count                 0
harsh_brake_count              0
harsh_accel_count              0
sharp_turn_count               0
stationary_pct                 0
gps_poor_quality_pct           0
acc_x_missing_pct              0
age                            0
driving_experience             0
driver_rating                  0
gender                         0
car_model_year                 0
driver_id 

---

**CATEGORY 5: DERIVED RATE FEATURES**

`PURPOSE`: Normalize counts by trip duration

`WHY RATES INSTEAD OF COUNTS?`
- A 5-minute trip with 2 harsh brakes ≠ 2-hour trip with 2 harsh brakes
- Rates account for "opportunity" to exhibit risky behavior
- More comparable across different trip lengths

`FEATURES TO CREATE:`  
18. speeding_rate: speeding_count / trip_duration
19. harsh_brake_rate: harsh_brake_count / trip_duration
20. sharp_turn_rate: sharp_turn_count / trip_duration

In [78]:
# Avoid division by zero
if (trip_features_ml['trip_duration'] == 0).any():
    print("\nFound trips with 0 duration. Replacing with 1...")
    trip_features_ml['trip_duration'] = trip_features_ml['trip_duration'].replace(0, 1)


# Feature 18-20: Event rates
print("\n18) speeding_rate (events per second)")
trip_features_ml['speeding_rate'] = (
    trip_features_ml['speeding_count'] / trip_features_ml['trip_duration']
)

print("19) harsh_brake_rate (events per second)")
trip_features_ml['harsh_brake_rate'] = (
    trip_features_ml['harsh_brake_count'] / trip_features_ml['trip_duration']
)

print("20) sharp_turn_rate (events per second)")
trip_features_ml['sharp_turn_rate'] = (
    trip_features_ml['sharp_turn_count'] / trip_features_ml['trip_duration']
)



18) speeding_rate (events per second)
19) harsh_brake_rate (events per second)
20) sharp_turn_rate (events per second)


In [79]:
# Show sample rates
print("\nRate feature statistics:")
rate_stats = trip_features_ml[['speeding_rate', 'harsh_brake_rate', 'sharp_turn_rate']].describe()
print(rate_stats)


Rate feature statistics:
       speeding_rate  harsh_brake_rate  sharp_turn_rate
count   19923.000000      19923.000000     19923.000000
mean        0.008883          0.001176         0.007303
std         0.028491          0.001899         0.021265
min         0.000000          0.000000         0.000000
25%         0.000000          0.000000         0.000000
50%         0.000000          0.000000         0.000708
75%         0.000000          0.001781         0.004622
max         0.507177          0.043689         0.402529


---

**CATEGORY 6: DRIVER-LEVEL HISTORICAL FEATURES**

`PURPOSE`:  Capture driver's past behavior patterns

`CRITICAL: AVOID DATA LEAKAGE`
- Must NOT include current trip in driver aggregates
- In production:  only past trips available
- Solution:  Exclude current trip from driver danger rate calculation

`FEATURES TO CREATE`:

21. driver_trip_count: How many trips does this driver have?

    → New drivers (1-2 trips) might behave differently

<br>

22. driver_danger_history_rate: % of driver's OTHER trips that were dangerous

    → Past behavior predicts future behavior (excluding current trip!)

Note: These are SIMPLE versions. In production with timestamps, 
we'd use time-aware features (only past trips).

In [80]:
# Feature 21: Driver trip count
print("\n21) driver_trip_count")

driver_trip_counts = trip_features_ml.groupby('driver_id').size().rename('driver_trip_count')
trip_features_ml = trip_features_ml.merge(driver_trip_counts, on='driver_id', how='left')

print(f"   Trip count distribution:")
print(trip_features_ml['driver_trip_count'].describe())


# Feature 22: Driver danger rate (EXCLUDING current trip)
print("\n22) driver_danger_history_rate (excludes current trip)")

# Total dangerous trips per driver
driver_danger_total = trip_features_ml.groupby('driver_id')['label'].sum().rename('driver_danger_total')
trip_features_ml = trip_features_ml.merge(driver_danger_total, on='driver_id', how='left')

# For each trip, subtract itself if dangerous
# This prevents leakage: model can't see current label in driver history
trip_features_ml['driver_danger_history_rate'] = (
    (trip_features_ml['driver_danger_total'] - trip_features_ml['label']) / 
    (trip_features_ml['driver_trip_count'] - 1).replace(0, 1)  # Avoid division by zero for single-trip drivers
)

# For drivers with only 1 trip, set to 0 (no history)
trip_features_ml.loc[trip_features_ml['driver_trip_count'] == 1, 'driver_danger_history_rate'] = 0

# Drop temporary column
trip_features_ml = trip_features_ml.drop(columns=['driver_danger_total'])

print(f"   Danger history rate distribution:")
print(trip_features_ml['driver_danger_history_rate'].describe())


21) driver_trip_count
   Trip count distribution:
count    19923.000000
mean        40.844501
std          6.319737
min         23.000000
25%         37.000000
50%         41.000000
75%         45.000000
max         59.000000
Name: driver_trip_count, dtype: float64

22) driver_danger_history_rate (excludes current trip)
   Danger history rate distribution:
count    19923.000000
mean         0.250414
std          0.068936
min          0.055556
25%          0.200000
50%          0.244444
75%          0.295455
max          0.545455
Name: driver_danger_history_rate, dtype: float64


In [81]:
trip_features_ml.columns

Index(['bookingID', 'trip_duration', 'avg_speed', 'max_speed', 'speed_std',
       'speed_p95', 'speed_p99', 'long_acc_mean', 'long_acc_std',
       'long_acc_min', 'long_acc_max', 'gyro_z_std', 'gyro_z_max',
       'gyro_z_abs_p95', 'speed_roll30_mean_max', 'speed_roll30_std_max',
       'acc_roll10_std_max', 'gyro_z_roll10_absmax_max', 'speeding_count',
       'harsh_brake_count', 'harsh_accel_count', 'sharp_turn_count',
       'stationary_pct', 'gps_poor_quality_pct', 'acc_x_missing_pct', 'age',
       'driving_experience', 'driver_rating', 'gender', 'car_model_year',
       'driver_id', 'label', 'max_consec_speeding', 'max_consec_harsh_brakes',
       'max_consec_sharp_turns', 'highspeed_brake_interaction',
       'speed_turn_interaction', 'volatility_interaction',
       'aggressive_burst_score', 'speeding_rate', 'harsh_brake_rate',
       'sharp_turn_rate', 'driver_trip_count', 'driver_danger_history_rate'],
      dtype='object')

In [83]:
# Creating harsh_accel_rate due to inconsistency
# Feature 23:  Harsh acceleration rate
trip_features_ml['harsh_accel_rate'] = (
    trip_features_ml['harsh_accel_count'] / trip_features_ml['trip_duration']
)

In [86]:
trip_features_ml.columns

Index(['bookingID', 'trip_duration', 'avg_speed', 'max_speed', 'speed_std',
       'speed_p95', 'speed_p99', 'long_acc_mean', 'long_acc_std',
       'long_acc_min', 'long_acc_max', 'gyro_z_std', 'gyro_z_max',
       'gyro_z_abs_p95', 'speed_roll30_mean_max', 'speed_roll30_std_max',
       'acc_roll10_std_max', 'gyro_z_roll10_absmax_max', 'speeding_count',
       'harsh_brake_count', 'harsh_accel_count', 'sharp_turn_count',
       'stationary_pct', 'gps_poor_quality_pct', 'acc_x_missing_pct', 'age',
       'driving_experience', 'driver_rating', 'gender', 'car_model_year',
       'driver_id', 'label', 'max_consec_speeding', 'max_consec_harsh_brakes',
       'max_consec_sharp_turns', 'highspeed_brake_interaction',
       'speed_turn_interaction', 'volatility_interaction',
       'aggressive_burst_score', 'speeding_rate', 'harsh_brake_rate',
       'sharp_turn_rate', 'driver_trip_count', 'driver_danger_history_rate',
       'harsh_accel_rate'],
      dtype='object')

In [88]:
trip_features_ml.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19923 entries, 0 to 19922
Data columns (total 45 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   bookingID                    19923 non-null  int64  
 1   trip_duration                19923 non-null  float64
 2   avg_speed                    19923 non-null  float64
 3   max_speed                    19923 non-null  float64
 4   speed_std                    19923 non-null  float64
 5   speed_p95                    19923 non-null  float64
 6   speed_p99                    19923 non-null  float64
 7   long_acc_mean                19923 non-null  float64
 8   long_acc_std                 19923 non-null  float64
 9   long_acc_min                 19923 non-null  float64
 10  long_acc_max                 19923 non-null  float64
 11  gyro_z_std                   19923 non-null  float64
 12  gyro_z_max                   19923 non-null  float64
 13  gyro_z_abs_p95  

---

**STEP 9:  FINAL FEATURE SUMMARY & SAVE**

In [89]:
# Count all features
# --------------------------------------------------
print(f"\nFINAL DATASET:")
print(f"   Total rows (trips): {trip_features_ml.shape[0]: ,}")
print(f"   Total columns: {trip_features_ml.shape[1]}")

# Count advanced features created
advanced_features = [
    'speed_roll30_mean_max', 'speed_roll30_std_max', 'acc_roll10_std_max', 
    'gyro_z_roll10_absmax_max',  # Rolling (4)
    'speed_p95', 'speed_p99', 'long_acc_min', 'long_acc_max', 
    'speed_std', 'gyro_z_abs_p95',  # Distribution (6)
    'max_consec_speeding', 'max_consec_harsh_brakes', 'max_consec_sharp_turns',  # Burstiness (3)
    'highspeed_brake_interaction', 'speed_turn_interaction', 
    'volatility_interaction', 'aggressive_burst_score',  # Interaction (4)
    'speeding_rate', 'harsh_brake_rate', 'sharp_turn_rate',  # Rates (3)
    'driver_trip_count', 'driver_danger_history_rate'  # Driver history (2)
]

print(f"\nADVANCED FEATURES CREATED:  {len(advanced_features)}")

# Verify all features exist
missing_features = [f for f in advanced_features if f not in trip_features_ml.columns]
if missing_features: 
    print(f"\nWARNING: Expected features not found:")
    for f in missing_features:
        print(f"   - {f}")
else:
    print(f"\nAll {len(advanced_features)} advanced features verified!")


# Data Quality Final Check
# --------------------------------------------------
print("\n" + "="*70)
print("DATA QUALITY FINAL CHECK")
print("="*70)

print(f"\nMissing values:  {trip_features_ml.isnull().sum().sum()}")
print(f"Infinite values: {np.isinf(trip_features_ml.select_dtypes(include=[np.number])).sum().sum()}")
print(f"Duplicate bookingIDs: {trip_features_ml.duplicated('bookingID').sum()}")

# Check label distribution
print(f"\nLABEL DISTRIBUTION:")
print(trip_features_ml['label'].value_counts())
print(f"   Danger rate: {trip_features_ml['label']. mean()*100:.2f}%")
print(f"   Imbalance ratio: {(trip_features_ml['label']==0).sum() / (trip_features_ml['label']==1).sum():.2f}: 1")


FINAL DATASET:
   Total rows (trips):  19,923
   Total columns: 45

ADVANCED FEATURES CREATED:  22

All 22 advanced features verified!

DATA QUALITY FINAL CHECK

Missing values:  0
Infinite values: 0
Duplicate bookingIDs: 0

LABEL DISTRIBUTION:
label
0    14934
1     4989
Name: count, dtype: int64
   Danger rate: 25.04%
   Imbalance ratio: 2.99: 1


In [ ]:
# Save Dataset
print("\n" + "="*70)
print("SAVING FEATURE-ENGINEERED DATASET")
print("="*70)

# Save as CSV
output_csv = "analysis/csv/trip_features_ml_advanced.csv"
trip_features_ml.to_csv(output_csv, index=False)
print(f"\nSaved:  {output_csv}")
print(f"   Size: ~{trip_features_ml.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

# Save as pickle (faster loading)
output_pkl = "datasets/trip_features_ml_advanced.pkl"
trip_features_ml.to_pickle(output_pkl)
print(f"Saved: {output_pkl}")


SAVING FEATURE-ENGINEERED DATASET

Saved:  datasets/trip_features_ml_advanced.csv
   Size: ~7.8 MB
Saved: datasets/trip_features_ml_advanced.pkl
